In [1]:
import torch
from torch.utils.data import DataLoader
import time
import copy
import pandas as pd
import os
from tqdm import tqdm
import itertools
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from L_calc_new import *

Running on cpu.
Model weights loaded successfully


Using cache found in C:\Users\alepa/.cache\torch\hub\chenyaofo_pytorch-cifar-models_master


Files already downloaded and verified


# Models

In [2]:
class LeNet():
    def __init__(self, weigths_pth = "models/lenet5_model.pth"):
        self.query_count = 0
        self.batch_size = 64
        self.num_classes = 10
        self.classes = [str(i) for i in range(10)]
        self.learning_rate = 0.001
        self.num_epochs = 10
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.seed = 42
        self.trained_mdl_pth = weigths_pth
        random.seed(self.seed)
        torch.manual_seed(self.seed)
        self.requires_denorm = True
        self.mean = [0.1307]
        self.std = [0.3081]

        self.transform=transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize((0.1307,), (0.3081,)),
                    ])
        
        self.testloader = torch.utils.data.DataLoader(
            datasets.MNIST('../data', train=False, download=True, transform=transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize((0.1307,), (0.3081,)),
                    ])),
                batch_size=1, shuffle=True)

        self.model = torch_model().to(self.device)
        self.model.load_state_dict(torch.load(self.trained_mdl_pth, map_location=self.device))
        self.model.eval()
        self.cost = nn.CrossEntropyLoss()


    def test(self):
        with torch.no_grad(): # note this should not be left on for WB attacks
            correct = 0
            total = 0
            for images, labels in self.test_loader:
                images = images.to(self.device)
                labels = labels.to(self.device)
                outputs = self.model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

            print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))


    def denorm(self, batch):
        mean, std = self.mean, self.std
        if isinstance(mean, list):
            mean = torch.tensor(mean).to(self.device)
        if isinstance(std, list):
            std = torch.tensor(std).to(self.device)
        return batch * std.view(1, -1, 1, 1) + mean.view(1, -1, 1, 1)

    def renorm(self, batch):
        return transforms.Normalize(tuple(self.mean), tuple(self.std))(batch).detach()

    def remake_testloader(self, seed):
        torch.manual_seed(seed)
        random.seed(seed)
        np.random.seed(seed)
        self.testloader = torch.utils.data.DataLoader(
            datasets.MNIST('../data', train=False, download=True, transform=transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize((0.1307,), (0.3081,)),
                    ])),
                batch_size=1, shuffle=True)
        return self.testloader
class torch_model(nn.Module):
    def __init__(self):
        super(torch_model, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [3]:
class simple_FashionMNIST():
    def __init__(self, trained_mdl_pth):
        self.query_count = 0
        self.trained_mdl_pth = trained_mdl_pth
        torch.manual_seed(42)
        np.random.seed(42)

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Running on {self.device}.")

        transform = transforms.Compose([transforms.ToTensor()])

        trainset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)

        self.testset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
        self.testloader = torch.utils.data.DataLoader(self.testset, batch_size=1, shuffle=False)
        self.requires_denorm = False

        self.classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
                'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot')
        self.num_classes = len(self.classes)

        self.model = BasicCNN().to(self.device)
        self.cost = nn.CrossEntropyLoss()

        if os.path.exists(self.trained_mdl_pth):
            self.load_model()
        else:
            print(f"No model file found at {self.trained_mdl_pth}.\nTraining...")
            self.train(self.trained_mdl_pth)

    def load_model(self, mdl_pth=None):
        mdl_pth = self.trained_mdl_pth if mdl_pth is None else mdl_pth
        #self.model = BasicCNN(self.num_classes).to(self.device)
        self.model.load_state_dict(torch.load(self.trained_mdl_pth))
        self.model.eval()
        print("Model weights loaded successfully")
        return
    
    def train(self):
        num_epochs = 5

        for epoch in range(num_epochs):
            self.model.train()
            
            for images, labels in self.trainloader:
                images, labels = images.to(self.device), labels.to(self.device)
                
                self.optimizer.zero_grad()
                
                outputs = self.model(images)
                
                loss = self.criterion(outputs, labels)
                
                loss.backward()
                
                self.optimizer.step()
    def denorm(self, batch):
        return batch
    def renorm(self, batch):
        return batch.detach()
    
    def remake_testloader(self, seed):
        torch.manual_seed(seed)
        random.seed(seed)
        np.random.seed(seed)
        self.testloader = torch.utils.data.DataLoader(self.testset, batch_size=1, shuffle=False)
        return self.testloader
class BasicCNN(nn.Module):
    def __init__(self):
        super(BasicCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)  # Output: [batch_size, 32, 26, 26]
        self.conv2 = nn.Conv2d(32, 64, 3) # Output: [batch_size, 64, 11, 11]
        
        self.fc1 = nn.Linear(64 * 5 * 5, 128)  # Flattening: [batch_size, 64*5*5]
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        
        x = x.view(-1, 64 * 5 * 5) # Flattening
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [4]:
class ResNet20():
    def __init__(self):
        self.query_count = 0
        self.model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_resnet20", pretrained=True)
        self.model.eval()
        self.num_classes = 10
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.seed = 42
        random.seed(self.seed)
        torch.manual_seed(self.seed)

        self.classes = [
                        'airplane', 										
                        'automobile', 									
                        'bird', 									
                        'cat', 										
                        'deer',										
                        'dog', 										
                        'frog', 										
                        'horse', 										
                        'ship', 										
                        'truck'
                        ]

        self.cost = nn.CrossEntropyLoss()
        
        self.mean = (0.4914, 0.4822, 0.4465)
        self.std = (0.2023, 0.1994, 0.2010)
        self.testset = torchvision.datasets.CIFAR10(root='./data', 
                                                train=False, 
                                                download=True,
                                                transform=transforms.Compose([
                                                    transforms.ToTensor(),
                                                    transforms.Normalize(self.mean, self.std)
                                                ])
        )
        self.testloader = torch.utils.data.DataLoader(self.testset,
                                                batch_size=1, 
                                                shuffle=True)
    
    def remake_testloader(self, seed):
        torch.manual_seed(seed)
        random.seed(seed)
        np.random.seed(seed)
        self.testloader = torch.utils.data.DataLoader(self.testset,
                                                batch_size=1, 
                                                shuffle=True)
        return self.testloader
    def denorm(self, batch):
        mean, std = self.mean, self.std
        mean = torch.tensor(mean).to(self.device)
        std = torch.tensor(std).to(self.device)
        return batch * std.view(1, -1, 1, 1) + mean.view(1, -1, 1, 1)
    
    def renorm(self, batch):
        return transforms.Normalize(self.mean, self.std)(batch).detach()

# Attack utils

In [5]:
class AttackStep:
    def __init__(self, method, epsilon, x0_denorm, lmo, stepsize_method=None, momentum=0.8):
        self.method = method
        self.epsilon = epsilon
        self.x0_denorm = x0_denorm
        self.stepsize_method = stepsize_method
        self.lmo = lmo
        self.momentum = momentum
        self.m_t_last = None
        self.m_t = None
        self.S_t = [x0_denorm]
        self.A_t = [1]
        self.x_t_denorm = None

    def step(self, x_t_denorm, x_t_grad):
        if self.method == 'fgsm':
            return self.fgsm_attack(x_t_denorm, x_t_grad)
        elif self.method == 'fw':
            return self.fw_step(x_t_denorm, x_t_grad)
        elif self.method == 'fw_momentum':
            return self.fw_step_momentum(x_t_denorm, x_t_grad, momentum=self.momentum)
        elif self.method == 'fw_away':
            return self.fw_step_away(x_t_denorm, x_t_grad)
        elif self.method == 'fw_away_m':
            return self.fw_step_away_m(x_t_denorm, x_t_grad, momentum=self.momentum)
        elif self.method == 'fw_pair':
            return self.fw_step_pairwise(x_t_denorm, x_t_grad)
        elif self.method == 'fw_pair_test':
            return self.fw_step_pairwise_test(x_t_denorm, x_t_grad)
        else:
            raise ValueError(f"Unknown method: {self.method}")
        
    def fgsm_attack(self, image, data_grad):
        # Collect the element-wise sign of the data gradient
        sign_data_grad = data_grad.sign()
        # Create the perturbed image by adjusting each pixel of the input image
        perturbed_image = image + self.epsilon * sign_data_grad
        # Adding clipping to maintain [0,1] range
        perturbed_image = torch.clamp(perturbed_image, 0, 1)
        # Return the perturbed image
        return perturbed_image

    def pgd_attack(self, x_t, g_t):
        perturbed_image = x_t + self.stepsize_method.get_stepsize(x_t, g_t) * g_t
        perturbed_image = torch.clamp(perturbed_image, 0, 1)
        return perturbed_image

    def fw_step(self, x_t, g_t):
        info = {}
        # Use LMO to compute the attack direction
        v_t = self.lmo.get(g_t)
        d_t = v_t - x_t
        self.d_t = d_t
        fw_stepsize = self.stepsize_method.get_stepsize(x_t, d_t)
        info['stepsize'] = fw_stepsize
        perturbed_image = x_t + fw_stepsize * d_t
        perturbed_image = torch.clamp(perturbed_image, 0, 1)
        gap_FW = torch.sum(-d_t * g_t).item()
        return perturbed_image, gap_FW, info

    def fw_step_momentum(self, x_t, g_t,  momentum=0.8):
        # alg from attacks.pdf
        info = {}
        m_t = (1 - momentum) * g_t
        if self.m_t_last is not None:
            m_t += momentum * self.m_t_last
        v_t = self.lmo.get(m_t)
        d_t = v_t - x_t

        fw_stepsize = self.stepsize_method.get_stepsize(x_t, d_t)

        gap_FW = torch.sum(-d_t * g_t).item()
        perturbed_image = x_t + fw_stepsize * d_t
        perturbed_image = torch.clamp(perturbed_image, 0, 1)
        self.m_t_last = m_t.clone().detach()
        return perturbed_image, gap_FW, info

    def update_active_away(self, gamma, s_t, v_t_idx, step_type, debug = True):
        """
        Args:
            gamma (float): stepsize
            gamma_max (int): Max stepsize informs when FW step will make S_t singular or AS dropstep
            S_t (list(torch.Tensor)): Active set of directions s.t. x_t in conv{S_t}
            A_t (list(float)): coefficients corresponding to atoms in S_t. x_t = A_t .* S_t
            v_t_idx (int): index of away atom in S_t
        """
        info = {}
        debug_info = {}
        if step_type == 'FW':
            if abs(gamma - 1) < 0.0001:
                # drop step
                self.S_t = [s_t]
                self.A_t = [1]
                debug_info['drop_step'] = 'FW'
            else:
                ## UPDATE S
                # need to check if vertex is already in S
                diffs = [torch.sum(torch.abs(s_t - s)).item() for s in self.S_t]
                min_diff = min(diffs)
                arg = np.argmin(diffs)
                if min_diff < 0.9*self.epsilon:
                    # s_t already in S_t
                    s_t_idx = arg
                    debug_info['FW_revisit'] = True
                else:
                    self.S_t.append(s_t)
                    self.A_t.append(0.0)
                    s_t_idx = -1
                #debug_info["min_revisit_diff"] = min_diff
                
                ## UPDATE ALPHAS
                self.A_t = [(1 - gamma) * alpha for alpha in self.A_t]
                self.A_t[s_t_idx] += gamma
        elif step_type == 'AS':
            if False: #gamma >= gamma_max:
                # drop step: remove atom and alpha
                # logic changed to check if alpha is zeroed (see below)
                self.A_t.pop(v_t_idx)
                self.S_t.pop(v_t_idx)
                debug_info['drop_step'] = 'AS'
            else:
                ## UPDATE ALPHAS
                self.A_t = [(1 + gamma) * alpha for alpha in self.A_t]
                self.A_t[v_t_idx] -= gamma
        else:
            raise Exception("Step must be FW or AS")
        
        # Detect if the away step was a dropstep
        # done by seeing if any of the alphas where zeroed out.
        i = 0
        while i < len(self.A_t):
            alpha_i = self.A_t[i]
            if alpha_i <= 0:
                debug_info['drop_step'] = 'AS'
                self.A_t.pop(i)
                self.S_t.pop(i)
            else:
                i += 1

        debug_info['v_t_idx'] = v_t_idx
        if debug:
            info.update(debug_info)
        return self.S_t, self.A_t, info

    def fw_step_away(self, x_t, g_t, debug=True):
        # alg from FW_varients.pdf
        use_conv_comb_x_t = False
        info = {}
        debug_info = {}
        
        # FW direction
        s_t = self.lmo.get(g_t)
        d_t_FW = s_t - x_t
        # AWAY direction. From set of vertices already visited
        away_sign = 1
        away_costs = []
        for v in self.S_t:
            away_costs.append(torch.sum(away_sign * g_t * v).item()) 
        v_t_idx = np.argmax(away_costs) 
        v_t = self.S_t[v_t_idx]
        d_t_AWAY = x_t - v_t
        # check optimality (FW gap)
        gap_FW = torch.sum(-g_t * d_t_FW).item()
        gap_AWAY = torch.sum(-g_t * d_t_AWAY).item()
        info['gap_FW'] = gap_FW
        info['gap_AS'] = gap_AWAY
        debug_info['awayCosts'] = away_costs

        # check which direction is closer to the -gradient
        if (gap_FW >= gap_AWAY) or (len(self.S_t) == 1): # don't step away if only one vertex in S_t
            step_type = 'FW'
            d_t = d_t_FW
            max_step = 1
        else:
            step_type = 'AS'
            d_t = d_t_AWAY
            alpha_v_t = self.A_t[v_t_idx]
            max_step = 1 if alpha_v_t == 1 else alpha_v_t / (1 - alpha_v_t)  # avoid divide by zero when alpha = 1
        self.d_t = d_t
        info['step_type'] = step_type
        debug_info['max_step'] = max_step
        # determine stepsize according to rule
        fw_stepsize = self.stepsize_method.get_stepsize(x_t, d_t, max_step)
        info['stepsize'] = fw_stepsize

        self.S_t, self.A_t, update_info = self.update_active_away(fw_stepsize, s_t, v_t_idx, step_type,
                                                debug=debug)

        ## UPDATE x_t
        perturbed_image_step = x_t + fw_stepsize * d_t
        perturbed_image_alpha = sum([alpha * v for alpha, v in zip(self.A_t, self.S_t)])
        if use_conv_comb_x_t: # use x_t = A_t.T * S_t or x_t + gamma * d_t ?
            perturbed_image = perturbed_image_alpha
        else:
            perturbed_image = perturbed_image_step
        perturbed_image = torch.clamp(perturbed_image, 0, 1)

        # LOGGING and DEBUG info
        info['alphas'] = self.A_t
        debug_info['L_inf_step'] = torch.max(torch.abs(perturbed_image_step - self.x0_denorm)).item()
        debug_info['L_inf_alpha'] = torch.max(torch.abs(perturbed_image_alpha - self.x0_denorm)).item()
        alpha_np = ((perturbed_image_alpha).squeeze(0).permute(1, 2, 0).numpy()).clip(0,1)
        step_np = ((perturbed_image_step).squeeze(0).permute(1, 2, 0).numpy()).clip(0,1)
        debug_info['step_alpha_diffFactor'] = (alpha_np - step_np).sum() / self.epsilon
        info.update(update_info)
        if debug:
            info.update(debug_info)
        self.last_d = d_t
        return perturbed_image, gap_FW, info

    def fw_step_away_m(self, x_t, g_t, debug=True, momentum=0.8):
        # alg from FW_varients.pdf
        use_conv_comb_x_t = False
        info = {}
        debug_info = {}
        
        # FW direction
        self.m_t = (1 - momentum) * g_t + momentum * self.m_t
        s_t = self.lmo.get(self.m_t)
        d_t_FW = s_t - x_t
        # AWAY direction. From set of vertices already visited
        away_sign = 1
        away_costs = []
        for v in self.S_t:
            away_costs.append(torch.sum(away_sign * g_t * v).item()) 
        v_t_idx = np.argmax(away_costs) 
        v_t = self.S_t[v_t_idx]
        d_t_AWAY = x_t - v_t
        # check optimality (FW gap)
        gap_FW = torch.sum(-g_t * d_t_FW).item()
        gap_AWAY = torch.sum(-g_t * d_t_AWAY).item()
        info['gap_FW'] = gap_FW
        info['gap_AS'] = gap_AWAY
        debug_info['awayCosts'] = away_costs

        # check which direction is closer to the -gradient
        if (gap_FW >= gap_AWAY) or (len(self.S_t) == 1): # don't step away if only one vertex in S_t
            step_type = 'FW'
            d_t = d_t_FW
            max_step = 1
        else:
            step_type = 'AS'
            d_t = d_t_AWAY
            alpha_v_t = self.A_t[v_t_idx]
            max_step = 1 if alpha_v_t == 1 else alpha_v_t / (1 - alpha_v_t)  # avoid divide by zero when alpha = 1
        self.d_t = d_t
        info['step_type'] = step_type
        debug_info['max_step'] = max_step
        # determine stepsize according to rule
        fw_stepsize = self.stepsize_method.get_stepsize(self.stepsize_method.x_t_denorm, d_t, max_step)
        info['stepsize'] = fw_stepsize

        self.S_t, self.A_t, update_info = self.update_active_away(fw_stepsize, s_t, v_t_idx, step_type,
                                                debug=debug)

        ## UPDATE x_t
        perturbed_image_step = x_t + fw_stepsize * d_t
        perturbed_image_alpha = sum([alpha * v for alpha, v in zip(self.A_t, self.S_t)])
        if use_conv_comb_x_t: # use x_t = A_t.T * S_t or x_t + gamma * d_t ?
            perturbed_image = perturbed_image_alpha
        else:
            perturbed_image = perturbed_image_step
        perturbed_image = torch.clamp(perturbed_image, 0, 1)

        # LOGGING and DEBUG info
        info['alphas'] = self.A_t
        debug_info['L_inf_step'] = torch.max(torch.abs(perturbed_image_step - self.x0_denorm)).item()
        debug_info['L_inf_alpha'] = torch.max(torch.abs(perturbed_image_alpha - self.x0_denorm)).item()
        alpha_np = ((perturbed_image_alpha).squeeze(0).permute(1, 2, 0).numpy()).clip(0,1)
        step_np = ((perturbed_image_step).squeeze(0).permute(1, 2, 0).numpy()).clip(0,1)
        debug_info['step_alpha_diffFactor'] = (alpha_np - step_np).sum() / self.epsilon
        info.update(update_info)
        if debug:
            info.update(debug_info)
        self.last_d = d_t
        return perturbed_image, gap_FW, info

    def update_active_pair(self, gamma, s_t, v_t_idx, info):
        drop = True
        diffs = [torch.sum(torch.abs(s_t - s)).item() for s in self.S_t]  # [torch.max(torch.abs(s_t - s)).item() for s in S_t]
        min_diff = min(diffs)
        arg = np.argmin(diffs)
        if min_diff < 0.9 * self.epsilon:
            # s_t already in S_t
            s_t_idx = arg
        else:
            self.S_t.append(s_t)
            self.A_t.append(0.0)
            s_t_idx = -1
        #self.A_t = [a + gamma if i == s_t_idx else a - gamma for i, a in enumerate(self.A_t)]
        self.A_t[s_t_idx] += gamma
        self.A_t[v_t_idx] -= gamma
        
        tol = 0.001
        i=0
        while i < len(self.A_t):
            alpha_v_i = self.A_t[i]
            s_t_i = self.S_t[i]
            if abs(1 - alpha_v_i) < tol:
                self.A_t = [1.0]
                self.S_t = [s_t_i]
                break
            elif self.A_t[i] < tol:
                if drop:
                    self.A_t.pop(i)
                    self.S_t.pop(i)
                else:
                    pass
            else:
                i += 1
        info['A_t'] = copy.deepcopy(self.A_t)
        return self.S_t, self.A_t

    def fw_step_pairwise(self, x_t, g_t):
        use_conv_comb_x_t = True
        info = {}
        debug_info = {}

        # Using LMO to compute s_t
        s_t = self.lmo.get(g_t)
        d_t_FW = s_t - x_t

        # AWAY direction. From set of vertices already visited
        away_costs = []
        for v in self.S_t:
            away_costs.append(torch.sum(g_t * v).item())
        v_t_idx = np.argmax(away_costs)
        v_t = self.S_t[v_t_idx]
        alpha_v_t = self.A_t[v_t_idx]
        max_step = alpha_v_t
        d_t_AWAY = x_t - v_t
        self.away_costs = away_costs

        gap_FW = torch.sum(-g_t * d_t_FW).item()
        gap_AWAY = torch.sum(-g_t * d_t_AWAY).item()
        info['gap_FW'] = gap_FW
        info['gap_AS'] = gap_AWAY

        d_t = d_t_FW + d_t_AWAY #s_t - v_t
        self.d_t = d_t
        fw_stepsize = self.stepsize_method.get_stepsize(x_t, d_t, max_step)

        self.S_t, self.A_t = self.update_active_pair(fw_stepsize, s_t, v_t_idx, info)
        perturbed_image_step = x_t + fw_stepsize * d_t
        perturbed_image_alpha = sum([alpha * v for alpha, v in zip(self.A_t, self.S_t)])
        if use_conv_comb_x_t: 
            perturbed_image = perturbed_image_alpha
        else:
            perturbed_image = perturbed_image_step
        perturbed_image = torch.clamp(perturbed_image, 0, 1)

        info.update(debug_info)

        return perturbed_image, gap_FW, info
    
    def update_active_pair_test(self, gamma, s_t, v_t_idx, debug = True):
        """
        Args:
            gamma (float): stepsize
            gamma_max (int): Max stepsize informs when FW step will make S_t singular or AS dropstep
            S_t (list(torch.Tensor)): Active set of directions s.t. x_t in conv{S_t}
            A_t (list(float)): coefficients corresponding to atoms in S_t. x_t = A_t .* S_t
            v_t_idx (int): index of away atom in S_t
        """
        info = {}
        debug_info = {}
        if abs(gamma - 1) < 0.0001:
            # drop step
            self.S_t = [s_t]
            self.A_t = [1.0]
            debug_info['drop_step'] = 'FW'
        else:
            ## UPDATE S
            # need to check if vertex is already in S
            diffs = [torch.sum(torch.abs(s_t - s)).item() for s in self.S_t]
            min_diff = min(diffs)
            arg = np.argmin(diffs)
            debug_info['min_revisit_diff'] = min_diff
            if min_diff < 0.9*self.epsilon:
                # s_t already in S_t
                s_t_idx = arg
                debug_info['FW_revisit'] = True
            else:
                self.S_t.append(s_t)
                self.A_t.append(0.0)
                s_t_idx = -1
            #debug_info["min_revisit_diff"] = min_diff
            
            ## UPDATE ALPHAS
            #self.A_t = [(1 - gamma) * alpha for alpha in self.A_t]
            self.A_t[s_t_idx] += gamma
            if False: #gamma >= gamma_max:
                # drop step: remove atom and alpha
                # logic changed to check if alpha is zeroed (see below)
                self.A_t.pop(v_t_idx)
                self.S_t.pop(v_t_idx)
                debug_info['drop_step'] = 'AS'
            else:
                ## UPDATE ALPHAS
                #self.A_t = [(1 + gamma) * alpha for alpha in self.A_t]
                self.A_t[v_t_idx] -= gamma
        
        # Detect if the away step was a dropstep
        # done by seeing if any of the alphas where zeroed out.
        i = 0
        while i < len(self.A_t):
            alpha_i = self.A_t[i]
            if (alpha_i <= 0) and (len(self.A_t) > 1):
                debug_info['drop_step'] = 'AS'
                self.A_t.pop(i)
                self.S_t.pop(i)
            else:
                i += 1

        debug_info['v_t_idx'] = v_t_idx
        if debug:
            info.update(debug_info)
        return self.S_t, self.A_t, info
    
    def fw_step_pairwise_test(self, x_t, g_t, debug=True):
        # alg from FW_varients.pdf
        use_conv_comb_x_t = True
        info = {}
        debug_info = {}
        
        # FW direction
        s_t = self.lmo.get(g_t)
        d_t_FW = s_t - x_t
        # AWAY direction. From set of vertices already visited
        away_sign = 1
        away_costs = []
        for v in self.S_t:
            away_costs.append(torch.sum(away_sign * g_t * v).item()) 
        v_t_idx = np.argmax(away_costs) 
        v_t = self.S_t[v_t_idx]
        d_t_AWAY = x_t - v_t
        # check optimality (FW gap)
        gap_FW = torch.sum(-g_t * d_t_FW).item()
        gap_AWAY = torch.sum(-g_t * d_t_AWAY).item()
        info['gap_FW'] = gap_FW
        info['gap_AS'] = gap_AWAY
        debug_info['awayCosts'] = away_costs

        # # check which direction is closer to the -gradient
        # if (gap_FW >= gap_AWAY) or (len(self.S_t) == 1): # don't step away if only one vertex in S_t
        #     step_type = 'FW'
        #     d_t = d_t_FW
        #     max_step = 1
        # else:
        #     step_type = 'AS'
        #     d_t = d_t_AWAY
        #     alpha_v_t = self.A_t[v_t_idx]
        #     max_step = 1 if alpha_v_t == 1 else alpha_v_t / (1 - alpha_v_t)  # avoid divide by zero when alpha = 1
        # info['step_type'] = step_type
        # debug_info['max_step'] = max_step
        # DEFINE PAIRWISE DIRECTION
        d_t = s_t - v_t
        alpha_v_t = self.A_t[v_t_idx]
        max_step = alpha_v_t
        info['max_stepsize'] = max_step
        # determine stepsize according to rule
        fw_stepsize = self.stepsize_method.get_stepsize(x_t, d_t, max_step)
        info['stepsize'] = fw_stepsize

        self.S_t, self.A_t, update_info = self.update_active_pair_test(fw_stepsize, s_t, v_t_idx,
                                                debug=debug)

        ## UPDATE x_t
        perturbed_image_step = x_t + fw_stepsize * d_t
        perturbed_image_alpha = sum([alpha * v for alpha, v in zip(self.A_t, self.S_t)])
        if use_conv_comb_x_t: # use x_t = A_t.T * S_t or x_t + gamma * d_t ?
            perturbed_image = perturbed_image_alpha
        else:
            perturbed_image = perturbed_image_step
        perturbed_image = torch.clamp(perturbed_image, 0, 1)

        # LOGGING and DEBUG info
        info['alphas'] = self.A_t
        debug_info['L_inf_step'] = torch.max(torch.abs(perturbed_image_step - self.x0_denorm)).item()
        debug_info['L_inf_alpha'] = torch.max(torch.abs(perturbed_image_alpha - self.x0_denorm)).item()
        alpha_np = ((perturbed_image_alpha).squeeze(0).permute(1, 2, 0).numpy()).clip(0,1)
        step_np = ((perturbed_image_step).squeeze(0).permute(1, 2, 0).numpy()).clip(0,1)
        debug_info['step_alpha_diffFactor'] = (alpha_np - step_np).sum() / self.epsilon
        info.update(update_info)
        if debug:
            info.update(debug_info)
        self.last_d = d_t
        return perturbed_image, gap_FW, info

# Other Utils

In [6]:
class LMO:
    def __init__(self, epsilon, x0, p):
        self.x0 = x0.clone().detach()  # Ensure x0 is not modified elsewhere
        self.epsilon = epsilon
        self.p = p
        # Select the appropriate LMO method based on the norm p
        if p == -1:
            self.method = self._LMO_inf
        elif p == 1:
            self.method = self._LMO_l1
        elif p == 2:
            self.method = self._LMO_l2
        else:
            raise Exception(f"invalid choice of norm {p}")

    def get(self, g_t):
        return self.method(g_t)

    def _LMO_inf(self, g_t):
        g_t_sign = g_t.sign()  # Get the sign of the gradient
        s_t = -self.epsilon * g_t_sign + self.x0  # Update step for l_inf norm
        return s_t

    def _LMO_l1(self, gradient):
        abs_gradient = gradient.abs()  # Get the absolute value of the gradient
        sign_gradient = gradient.sign()  # Get the sign of the gradient
        perturbation = torch.zeros_like(gradient)
        # For each example in the batch, select the component with the maximum absolute gradient
        for i in range(gradient.size(0)):
            _, idx = torch.topk(abs_gradient[i].view(-1), 1)
            perturbation[i].view(-1)[idx] = sign_gradient[i].view(-1)[idx]
        return self.epsilon * perturbation

    def _LMO_l2(self, g_t): # from arxiv version of attacks.pdf
        g_t_norm = torch.norm(g_t, p=2, dim=-1, keepdim=True)
        s_t = -self.epsilon * g_t / g_t_norm + self.x0
        return s_t

class AdversarialLoss(nn.Module):
    def __init__(self, num_classes, specific_label=None):
        """
        Initialize the AdversarialLoss.
        
        Args:
        - num_classes (int): Total number of classes in the classification problem.
        - specific_label (int, optional): A specific incorrect label to target. If None, the loss will consider all incorrect labels.
        """
        super(AdversarialLoss, self).__init__()
        self.num_classes = num_classes
        self.specific_label = specific_label

    def forward(self, outputs, targets):
        """
        Compute the adversarial loss.
        
        Args:
        - outputs (torch.Tensor): The model outputs (logits) of shape (batch_size, num_classes).
        - targets (torch.Tensor): The true labels of shape (batch_size,).
        
        Returns:
        - loss (torch.Tensor): The computed adversarial loss.
        """
        batch_size = outputs.size(0)
        if self.specific_label is not None:
            # Targeting a specific incorrect label
            incorrect_labels = torch.full_like(targets, self.specific_label)
            mask = (incorrect_labels != targets).float()
            specific_log_probs = F.log_softmax(outputs, dim=1).gather(1, incorrect_labels.unsqueeze(1)).squeeze(1)
            loss = -specific_log_probs * mask
            return loss.mean()

        # if self.specific_label is not None:
        #     # Targeting a specific incorrect label
        #     specific_labels = torch.full_like(targets, self.specific_label)
            
        #     # Compute log probabilities
        #     log_probs = F.log_softmax(outputs, dim=1)
            
        #     # Get the log probabilities of the specific label
        #     specific_log_probs = log_probs.gather(1, specific_labels.unsqueeze(1)).squeeze(1)
            
        #     # Compute the negative log likelihood
        #     loss = -specific_log_probs
        #     return loss.mean()
        else:
            # Averaging over all incorrect labels
            log_probs = F.log_softmax(outputs, dim=1)
            incorrect_log_probs = log_probs.clone()
            correct_log_probs = log_probs.gather(1, targets.unsqueeze(1)).squeeze(1)
            for i in range(batch_size):
                incorrect_log_probs[i, targets[i]] = float('-inf')
            average_incorrect_log_probs = incorrect_log_probs.logsumexp(dim=1) - torch.log(torch.tensor(self.num_classes - 1, dtype=torch.float))
            loss = -average_incorrect_log_probs
            return loss.mean()
        
    def forward1(self, outputs, targets):
        """
        Compute the adversarial loss.
        
        Args:
        - outputs (torch.Tensor): The model outputs (logits) of shape (batch_size, num_classes).
        - targets (torch.Tensor): The true labels of shape (batch_size,).
        
        Returns:
        - loss (torch.Tensor): The computed adversarial loss.
        """
        batch_size = outputs.size(0)
        if self.specific_label is not None:
            adv_target = torch.full((outputs.size(0),), self.specific_label, dtype=torch.long)
            if isinstance(targets, int):
                targets = torch.full((outputs.size(0),), targets, dtype=torch.long)
            
            return F.cross_entropy(outputs, adv_target)
        else:
            if isinstance(targets, int):
                targets = torch.full((outputs.size(0),), targets, dtype=torch.long)
            return -F.nll_loss(outputs, targets)

class stepsize():
    def __init__(self, model, strat, x0, fixed_size = 1, ls_criterion=None, ls_target = None, ls_num_samples=50, renorm = None):
        if isinstance(strat, (float, int)):
            fixed_size = strat
            strat = 'fixed'
        self.model = model
        self.strat = strat
        self.fixed_size = fixed_size
        # used for amjo
        self.x0 = x0
        self.x_t_grad = None
        self.loss0 = None

        # used for ls
        self.ls_criterion = ls_criterion
        self.ls_target = ls_target
        self.ls_num_samples = ls_num_samples
        self.renorm = renorm
        
        self.L_MNIST = L_MNIST
        self.L_FMNIST = L_FMNIST
        self.L_CIFAR = L_CIFAR

        if self.strat == 'lipschitz_mnist':
            self.stepsize = 1 / self.L_MNIST
        elif self.strat == 'lipschitz_fmnist':
            self.stepsize = 1 / self.L_FMNIST
        elif self.strat == 'lipschitz_cifar':
            self.stepsize = 1 / self.L_CIFAR

        self.stepsize = fixed_size # will be updated if using other method
        if self.strat not in ['fixed', 'ls', 'decay', 'amjo', 'lipschitz_mnist', 'lipschitz_fmnist', 'lipschitz_cifar']:
            raise Exception("Accepted stepsize rules are ['fixed', 'ls', 'decay', 'amjo', 'lipschitz_mnist', 'lipschitz_fmnist', 'lipschitz_cifar']")
    
    def set_stepsize_decay(self, t):
        self.stepsize = 2 / (t + 2)
        return

    def stepsize_linesearch(self, x_t, d_t, max_step = 1):
        x_tc = x_t.clone().detach()
        d_tc = d_t.clone().detach()
        losses = []
        with torch.no_grad():
            steps = [max_step * (i + 1) / self.ls_num_samples for i in range(self.ls_num_samples)]
            self._sizes_ls = steps
            for step in steps:
                new_x = self.renorm(x_tc + step * d_tc)
                output = self.model(new_x)
                loss = self.ls_criterion(output, self.ls_target).item()
                losses.append(loss)
        best_idx = np.argmin(losses)
        self.stepsize = steps[best_idx]
        
        self._loss_ls = losses
        return self.stepsize

    def stepsize_armijo(self, x_t, d_t, max_step = 1):
        info_step = {}
        x_tc = x_t.clone().detach()
        d_tc = d_t.clone().detach()
        step_size = max_step
        best_stepsize = max_step
        gamma = 0.5
        delta = 0.5
        initial_loss = self.ls_criterion(self.model(self.renorm(x_tc)), self.ls_target)#F.cross_entropy(self.model(x_k), target).item()
        min_loss = float('inf')
        
        while step_size > 1e-4:
            new_point = self.renorm(x_tc + step_size * d_tc)
            new_loss = self.ls_criterion(self.model(new_point), self.ls_target)#F.cross_entropy(self.model(new_point), target).item()
            RHS = initial_loss + gamma * step_size * torch.sum(self.x_t_grad * d_tc).item()
            if new_loss < min_loss:
                min_loss = new_loss
                best_stepsize = step_size
            if new_loss <= RHS:
                return step_size
            
            step_size *= delta
        
        return best_stepsize


    def get_stepsize(self, x_t, d_t, max_step = 1):
        if self.strat == 'ls':
            #fw_stepsize = self.exact_ls(x_t, d_t, max_step)
            fw_stepsize = self.stepsize_linesearch(x_t, d_t, max_step)
        elif self.strat == 'amjo':
            fw_stepsize = self.stepsize_armijo(x_t, d_t, max_step)
        else:
            fw_stepsize = self.stepsize
        fw_stepsize = min(fw_stepsize, max_step)
        return fw_stepsize

def early_stopper(criterion, t, success, first_success, info, gap_FW_tol, max_fw_iter, gap_FW):
    new_correct = 0
    if (criterion == 'pred') and first_success:
        # The attack was successful so the classification was not correct
        stop = True
        stop_reason = 'pred'
    elif (criterion == 'gap_FW') and (gap_FW < gap_FW_tol):
        if not success: # attack failed
            new_correct +=1
        stop = True
        stop_reason = 'gap'
    elif (t == max_fw_iter - 1): # Stop condition: Hit max FW iters
        stop = True
        stop_reason = 'max_iter'
        if not success: # attack failed
            new_correct +=1
    elif (criterion == 'gap_pairwise') and (info is not None) and (info['gap_pairwise'] < gap_FW_tol):
        if not success: # attack failed
            new_correct +=1
        stop = True
        stop_reason = 'gap_pairwise'
    else:
        # no stop criteria met, continue
        stop = False
        stop_reason = None
    return stop, stop_reason, new_correct

class example_saver():
    def __init__(self, num_adv_ex = 10, num_failed_ex = 10) -> None:
        self.num_adv_ex = num_adv_ex
        self.num_failed_ex = num_failed_ex
        self.adv_true = []
        self.adv_pred = []
        self.adv_x0 = []
        self.adv_atk = []
        self.adv_xt = []
        self.adv_true_init_prob = []
        self.adv_final_prob = []
        self.adv_ex = []
        self.failed_ex = []
        self.info = []
        pass

    def save_ex(self, perturbed_image, x0, true, final_pred, success, true_class_prob0, pred_class_prob):
        self.info.append(true)
        
        if (len(self.adv_ex) >= self.num_adv_ex) and (len(self.failed_ex) >= self.num_failed_ex):
            return

        atk = perturbed_image - x0
        atk = atk.detach().squeeze().cpu().numpy()
        x0 = x0.detach().squeeze().cpu().numpy()
        ex = perturbed_image.squeeze().detach().cpu().numpy()
        if len(atk.shape) > 2:
            atk = np.transpose(atk, (1, 2, 0))
            x0 = np.transpose(x0, (1, 2, 0))
            ex = np.transpose(ex, (1, 2, 0))


        # Save some adv examples for visualization later
        if success and (len(self.adv_ex) < self.num_adv_ex):
            self.adv_ex.append( (true.item(), final_pred.item(), ex) )
            self.adv_true.append(true.item())
            self.adv_pred.append(final_pred.item())
            self.adv_x0.append(x0)
            self.adv_atk.append(atk)
            self.adv_true_init_prob.append(true_class_prob0)
            self.adv_final_prob.append(pred_class_prob)
            self.adv_xt.append(ex)
        if (not success) and (len(self.failed_ex) < self.num_failed_ex):
            self.failed_ex.append( (true.item(), final_pred.item(), ex) )

def test(target_model, device, epsilon,num_fw_iter, num_test = 1000, method='fw', early_stopping = None, fw_stepsize_rule = 1, gap_FW_tol = 0.05, targeted = False, ex_saver=None, norm_p=-1, seed=42):
    start = time.process_time()
    target_model.query_count = 0
    testloader = target_model.remake_testloader(seed)
    model = target_model.model

    # Accuracy counter
    correct = 0
    adv_examples = []
    hist = []
    ex_num = 0
    # Loop over all examples in test set
    for x0, target in tqdm(testloader):
        x_t = x0.detach().clone().to(device)  # Clone and move to device
        # Send the data and label to the device
        x0, target = x0.to(device), target.to(device)
        x0_denorm = target_model.denorm(x0)
        if targeted:
            # select a random target for attack that is not the true target.
            adv_target = random.randint(0, target_model.num_classes - 2)
            adv_target = adv_target if adv_target < target else adv_target + 1
            criterion = AdversarialLoss(target_model.num_classes, specific_label=adv_target)
        else:
            criterion = AdversarialLoss(target_model.num_classes)
        lmo = LMO(epsilon, x0_denorm, norm_p)
        stepsize_method = stepsize(model, fw_stepsize_rule, x0_denorm, ls_criterion=criterion, ls_target=target, renorm = target_model.renorm)
        attackStep = AttackStep(method, epsilon, x0_denorm, lmo, stepsize_method)
        #x_t.requires_grad = True  #Set requires_grad attribute of tensor. Important for Attack
        had_first_success = False
        gap_FW = None
        info = None
        true_class_prob0 = 0

        for t in range(num_fw_iter):
            # Step size calculation
            if stepsize_method.strat == 'decay':
                stepsize_method.set_stepsize_decay(t)
            x_t.requires_grad = True
            # Forward pass the data through the model
            output = model(x_t)
            target_model.query_count += 1
            class_probs = torch.softmax(output,dim=1)
            # Calculate the loss
            loss = criterion(output, target)

            if t==0:
                # save init confidence in true class
                true_class_prob0 = class_probs[0, target.item()].item()
                stepsize_method.loss0 = loss.item()
            init_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability

            # If the initial prediction is wrong, don't bother attacking, just move on
            if (init_pred.item() != target.item()) and (t == 0):
                ex_num -= 1 # don't count this example
                break
            
            # Zero all existing gradients
            model.zero_grad()
            # Calculate gradients of model in backward pass
            loss.backward()
            x_t_grad = x_t.grad#.data
            # Restore the data to its original scale
            x_t_denorm = target_model.denorm(x_t)

            # save information needed for linesearching stepsize rules
            stepsize_method.x_t_grad = x_t_grad.clone().detach()

            # Call Attack
            with torch.no_grad():
                perturbed_image, gap_FW, info = attackStep.step(x_t_denorm, x_t_grad)
            
            # Reapply normalization
            x_t = target_model.renorm(perturbed_image)#transforms.Normalize((0.1307,), (0.3081,))(perturbed_image).detach()
            # Re-classify the perturbed image
            x_t.requires_grad = False
            output = model(x_t)
            info['l_inf'] = torch.max(torch.abs(x0_denorm - perturbed_image)).item()
            info['mdlLoss'] = loss.item()

            # Check for success
            final_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            if final_pred.item() == target.item():
                success = False
                first_success = False
            else:
                first_success =  not had_first_success
                had_first_success = True
                success = True
            stop, stop_reason, new_correct = early_stopper(early_stopping, t, success, first_success, info, gap_FW_tol, num_fw_iter, gap_FW)
            correct += new_correct

            # metric logging
            hist_iter = {
                'example_idx':ex_num,
                'FW_iter': t + 1, # original example is 0
                'gap_FW': gap_FW if gap_FW is not None else None,
                'success': success,
                'first_success': first_success,
                'target': target.item(),
                'pred': final_pred.item(),
                'stop_cond': stop_reason
            }
            if targeted:
                hist_iter['adv_target'] = adv_target
                targeted_success = (final_pred.item() == adv_target)
                info['targeted_success'] = targeted_success
            else:
                targeted_success = False
        
            if info is not None:
                hist_iter.update(info) # some methods output dict containing info at each step
            hist.append(hist_iter)
            if stop:
                class_probs = torch.softmax(output,dim=1)
                pred_class_prob = class_probs[0, final_pred.item()].item()
                if ex_saver is not None:
                    save_as_adv = targeted_success if targeted else success
                    ex_saver.save_ex(perturbed_image, x0_denorm, target, final_pred, save_as_adv, true_class_prob0, pred_class_prob)
                break
        ex_num += 1
        if ex_num >= num_test: # limit test set for speed
            break

    # Calculate final accuracy for this epsilon
    final_acc = correct/num_test
    end = time.process_time()
    print(f"Attack Method: {method}\tNorm: {norm_p}\tTargeted: {targeted}\tStep Rule: {fw_stepsize_rule}")
    print(f"Epsilon: {epsilon}\tCorrect Classifications (Failed Attacks) = {correct} / {num_test} = {final_acc}" + f"\tTime: {end - start:.2f}s")
    print(f"Total queries made: {target_model.query_count}")

    # Return the accuracy and an adversarial example
    return final_acc, adv_examples, pd.DataFrame(hist)

def display_examples(ex_saver, epsilon, classes, show_atk_mag = False, n_col = 3, offset = 2):
    # classes either int corresponding to number of classes, or list with class names
    if isinstance(classes, int):
        classes = list(range(classes))
    dim = len(ex_saver.adv_x0[0].shape)
    
    
    n_col = min(n_col, len(ex_saver.adv_pred))
    cmap = None if dim > 2 else 'gray'

    fig, axs = plt.subplots(3,n_col)
    for i in range(n_col):
        ex_idx = i + offset
        true = ex_saver.adv_true[ex_idx]
        pred = ex_saver.adv_pred[ex_idx]
        x0 = ex_saver.adv_x0[ex_idx]
        atk = (ex_saver.adv_atk[ex_idx]+epsilon)/(2*epsilon)
        if show_atk_mag:
            atk = np.abs(atk-0.5)
            atk *= 1/np.max(atk)
        atk = np.clip(atk,0,1)
        xt = ex_saver.adv_xt[ex_idx]
        prob_true = ex_saver.adv_true_init_prob[ex_idx]
        prob_adv = ex_saver.adv_final_prob[ex_idx]
        axs[0, i].imshow(x0, cmap = cmap)
        axs[0, i].axis('off')
        axs[0, i].set_title(f"Orginal: {classes[true]}\np = {prob_true:.2f}", fontsize=10)
        axs[1, i].imshow(atk, cmap = cmap)
        axs[1, i].axis('off')
        axs[1, i].set_title(f"Scaled offset:\nϵ = {epsilon}", fontsize=10)
        axs[2, i].imshow(xt, cmap = cmap)
        axs[2, i].axis('off')
        axs[2, i].set_title(f"Adv Pred: {classes[pred]}\np = {prob_adv:.2f}", fontsize=10)
    plt.tight_layout(pad=1.0, w_pad=-15, h_pad=1.0)
    plt.show()

# Attacks

In [7]:
epsilons = [0.005, 0.01, 0.05, 0.1, 0.25]
attack_types = ['fw', 'fw_away', 'fw_pair']
norm_types = [-1, 1, 2]
targeted_options = [True, False]

In [8]:
def run_experiment(epsilon, attack_type, norm_type, targeted, step_rule):
    ex_saver = example_saver()
    acc, ex, hist = test(target_mdl, device,
                         epsilon=epsilon,
                         num_fw_iter=20,
                         method=attack_type,
                         early_stopping='gap_FW' if attack_type != 'fw_away' else 'gap_AS',
                         fw_stepsize_rule=step_rule,
                         gap_FW_tol=0.1,
                         targeted=targeted,
                         num_test=100,
                         ex_saver=ex_saver,
                         norm_p=norm_type,
                         seed=11)
    
    total_queries = target_mdl.query_count
    
    result = {
        'epsilon': epsilon,
        'attack_type': attack_type,
        'norm_type': norm_type,
        'targeted': targeted,
        'step_rule': step_rule,
        'accuracy': acc,
        'total_queries': total_queries
    }
    
    return result

In [9]:
from models.LeNet import *
target_mdl = LeNet("models/lenet_mnist_model.pth")
model = target_mdl.model
device = target_mdl.device
test_loader = target_mdl.testloader
mdl_name='MNIST'

In [10]:
step_rules = ['lipschitz_mnist','decay', 'amjo', 'ls']
columns = ['epsilon', 'attack_type', 'norm_type', 'targeted', 'step_rule', 'accuracy', 'total_queries']
results_df = pd.DataFrame(columns=columns)


results_list = []

for epsilon, attack_type, norm_type, targeted, step_rule in itertools.product(epsilons, attack_types, norm_types, targeted_options, step_rules):
    result = run_experiment(epsilon, attack_type, norm_type, targeted, step_rule)
    results_list.append(result)
    
results_df = pd.concat([results_df, pd.DataFrame(results_list)], ignore_index=True)

results_df.to_csv('experiment_results.csv', index=False)

results_df

  1%|          | 101/10000 [00:04<07:14, 22.79it/s]


Attack Method: fw	Norm: -1	Targeted: True	Step Rule: lipschitz_mnist
Epsilon: 0.005	Correct Classifications (Failed Attacks) = 100 / 100 = 1.0	Time: 20.19s
Total queries made: 1948


  1%|          | 101/10000 [00:04<07:20, 22.45it/s]


Attack Method: fw	Norm: -1	Targeted: True	Step Rule: decay
Epsilon: 0.005	Correct Classifications (Failed Attacks) = 100 / 100 = 1.0	Time: 20.73s
Total queries made: 1948


  1%|          | 101/10000 [00:06<11:02, 14.93it/s]


...

Attack Method: fw_pair	Norm: 2	Targeted: False	Step Rule: amjo
Epsilon: 0.25	Correct Classifications (Failed Attacks) = 85 / 100 = 0.85	Time: 35.78s
Total queries made: 1856


  1%|          | 101/10000 [01:03<1:43:00,  1.60it/s]

Attack Method: fw_pair	Norm: 2	Targeted: False	Step Rule: ls
Epsilon: 0.25	Correct Classifications (Failed Attacks) = 85 / 100 = 0.85	Time: 268.11s
Total queries made: 1855



C:\Users\alepa\AppData\Local\Temp\ipykernel_43944\868000008.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame(results_list)], ignore_index=True)


,epsilon,attack_type,norm_type,targeted,step_rule,accuracy,total_queries
0,0.005,fw,-1,True,lipschitz_mnist,1.00,1948
1,0.005,fw,-1,True,decay,1.00,1948
2,0.005,fw,-1,True,amjo,1.00,1948
3,0.005,fw,-1,True,ls,1.00,1948
4,0.005,fw,-1,False,lipschitz_mnist,1.00,1876
...,...,...,...,...,...,...,...
355,0.250,fw_pair,2,True,ls,0.95,2002
356,0.250,fw_pair,2,False,lipschitz_mnist,0.88,1928
357,0.250,fw_pair,2,False,decay,0.85,1859
358,0.250,fw_pair,2,False,amjo,0.85,1856


In [11]:
from models.simple_FashionMNIST import *
target_mdl = simple_FashionMNIST("models/simple_FashionMNIST.pth")
model = target_mdl.model
device = target_mdl.device
test_loader = target_mdl.testloader
mdl_name = 'FMNIST'

Running on cpu.
Model weights loaded successfully
